In [23]:
from dotenv import load_dotenv 
from groq import Groq 
import os 
from IPython.display import display_markdown
from agentic_patterns import ReflectionAgent


class FixedReflectionAgent(ReflectionAgent):
    def __init__(self,model='llama-70b-8192',reflection_model='llama-70b-8192'):
        super().__init__()
        self.model=model
        self.reflection_model=reflection_model

    def generate(self, generation_history:list, verbose:int = 0):
        try:
            response=self.client.chat.completions.create(
                messages=generation_history,
                model=self.model,
                temperature=0.0,
                max_tokens=4000,
            )
            return response.choices[0].message.content
        except Exception as e:
            if verbose >=1:
                print(f'Error during genration: {e}')
            raise

    def reflect(self, reflection_history, verbose = 0):
        try:
             response=self.client.chat.completions.create(
                 messages=reflection_history,
                 model=self.reflection_model,
                 temperature=0.0,
                 max_tokens=4000

             )
             return response.choices[0].message.content
        except Exception as e: 
            if verbose >= 1:
                print(f"Error during refelction: {e}")
            raise

client=Groq(api_key=os.getenv('GROQ_API_KEY'))

In [24]:
#GENERATE FIRST VERSION

generate_chat_history=[
    {

        'role':'system',
        'content': "You are a Python programmer tasked with generating high quality Python code."
        "Respond only with the code implementation, no explanations."

    },
    {
        'role':'user',
        'content':'Generate a Python implementation of the Merge Sort algorithm'
    }
]
    

In [25]:
response=client.chat.completions.create(
    messages=generate_chat_history,
    model='llama3-70b-8192',
    temperature=0.0,
    max_tokens=1000

)
mergesort_code=response.choices[0].message.content
display_markdown(mergesort_code,raw=True)



```
def merge_sort(arr):
    if len(arr) <= 1:
        return arr
    mid = len(arr) // 2
    left_half = arr[:mid]
    right_half = arr[mid:]
    return merge(merge_sort(left_half), merge_sort(right_half))


def merge(left, right):
    merged = []
    left_index = 0
    right_index = 0
    while left_index < len(left) and right_index < len(right):
        if left[left_index] <= right[right_index]:
            merged.append(left[left_index])
            left_index += 1
        else:
            merged.append(right[right_index])
            right_index += 1
    merged.extend(left[left_index:])
    merged.extend(right[right_index:])
    return merged
```

In [26]:
# Reflection process
reflection_history = [
    {
        'role': 'system',
        'content': "You are Andrej Karpathy, an experienced computer scientist. "
                   "Provide concise technical critique of this Python code. Focus on: "
                   "1. Algorithm correctness 2. Code efficiency 3. Edge cases 4. Python best practices"
    },
    {
        'role': 'user',
        'content': f"Critique this merge sort implementation:\n\n{mergesort_code}"
    }
]

In [27]:
#get critique 

critique=client.chat.completions.create(
    messages=reflection_history,
    model='llama3-70b-8192',
    temperature=0.0,
    max_tokens=1000

).choices[0].message.content
print("\nCode Critique:")
display_markdown(critique, raw=True)


Code Critique:


Here's a concise technical critique of the provided merge sort implementation:

**Algorithm Correctness: 9/10**

The implementation is mostly correct, but there's a subtle bug in the `merge` function. In the comparison `if left[left_index] <= right[right_index]:`, it should be `if left[left_index] < right[right_index]:` to ensure stability of the sort. With the current implementation, if there are duplicate elements, their original order might not be preserved.

**Code Efficiency: 8/10**

The time complexity of the implementation is O(n log n), which is optimal for merge sort. However, the `merge` function can be optimized by using a single `append` call instead of extending the list with slices. This would reduce the number of operations and make the code more efficient.

**Edge Cases: 9/10**

The implementation correctly handles edge cases such as an empty input array or an array with a single element. However, it assumes that the input array only contains comparable elements. If the input array contains non-comparable elements (e.g., strings and integers), the implementation will raise a `TypeError`. It would be better to add a type check or a `try`-`except` block to handle such cases.

**Python Best Practices: 8/10**

The code is mostly readable and follows PEP 8 guidelines. However, there are a few areas for improvement:

* The function names `merge_sort` and `merge` could be more descriptive, e.g., `merge_sort_array` and `merge_sorted_arrays`.
* The variable names `left_half` and `right_half` could be more concise, e.g., `left` and `right`.
* The `merge` function could be simplified by using a list comprehension instead of explicit loops.
* There are no docstrings or comments to explain the purpose and behavior of the functions.

Here's an updated version of the implementation that addresses these issues:
```
def merge_sort_array(arr):
    """Sorts an array using the merge sort algorithm."""
    if len(arr) <= 1:
        return arr
    mid = len(arr) // 2
    left = arr[:mid]
    right = arr[mid:]
    return merge_sorted_arrays(merge_sort_array(left), merge_sort_array(right))


def merge_sorted_arrays(left, right):
    """Merges two sorted arrays into a single sorted array."""
    merged = []
    left_index = 0
    right_index = 0
    while left_index < len(left) and right_index < len(right):
        if left[left_index] < right[right_index]:
            merged.append(left[left_index])
            left_index += 1
        else:
            merged.append(right[right_index])
            right_index += 1
    merged.extend(left[left_index:])
    merged.extend(right[right_index:])
    return merged
```

In [28]:
#revised implimentation

generate_chat_history.append(
    {
        'role':'assistant',
        'content':mergesort_code
    }
)

generate_chat_history.append(
    {
        'role':'user',
        'content':f"Based on this critique, revise the implementation:\n\n{critique}"
}
)


In [29]:
revised_code=client.chat.completions.create(
    messages=generate_chat_history,
    model='llama3-70b-8192',
    temperature=0.0,
    max_tokens=1000
).choices[0].message.content
print("\n revised implementation")
display_markdown(revised_code,raw=True)


 revised implementation


```
def merge_sort_array(arr):
    """Sorts an array using the merge sort algorithm."""
    if len(arr) <= 1:
        return arr
    mid = len(arr) // 2
    left = arr[:mid]
    right = arr[mid:]
    return merge_sorted_arrays(merge_sort_array(left), merge_sort_array(right))


def merge_sorted_arrays(left, right):
    """Merges two sorted arrays into a single sorted array."""
    merged = []
    left_index = 0
    right_index = 0
    while left_index < len(left) and right_index < len(right):
        if left[left_index] < right[right_index]:
            merged.append(left[left_index])
            left_index += 1
        else:
            merged.append(right[right_index])
            right_index += 1
    merged.extend(left[left_index:])
    merged.extend(right[right_index:])
    return merged
```

In [30]:
## Reflection agent with fixed models
agent= FixedReflectionAgent(
    model='llama3-70b-8192',
    reflection_model='llama3-70b-8192'
)

final_response = agent.run(
    user_msg="Generate a production-quality Python implementation of the Merge Sort algorithm",
    generation_system_prompt="You are an expert Python developer. Generate efficient, well-commented code.",
    reflection_system_prompt="You are a senior software engineer reviewing code. Identify flaws and suggest improvements.",
    n_steps=3,
    verbose=1
)

print("\nFinal Refined Implementation:")
display_markdown(final_response, raw=True)



STEP 1/3

 

Stop Sequence found. Stopping the reflection loop ... 



Final Refined Implementation:


Here is a production-quality Python implementation of the Merge Sort algorithm:
```
def merge_sort(arr: list) -> list:
    """
    Sorts an array of elements using the Merge Sort algorithm.

    Time complexity: O(n log n)
    Space complexity: O(n)

    :param arr: The input array to be sorted
    :return: A new sorted array
    """
    # Base case: If the input array has only one element, it is already sorted
    if len(arr) <= 1:
        return arr

    # Split the input array into two halves
    mid = len(arr) // 2
    left = arr[:mid]
    right = arr[mid:]

    # Recursively sort the two halves
    left = merge_sort(left)
    right = merge_sort(right)

    # Merge the two sorted halves into a single sorted array
    return merge(left, right)


def merge(left: list, right: list) -> list:
    """
    Merges two sorted arrays into a single sorted array.

    :param left: The first sorted array
    :param right: The second sorted array
    :return: A new sorted array containing all elements from both input arrays
    """
    result = []
    i = j = 0

    # Merge smaller elements from both arrays
    while i < len(left) and j < len(right):
        if left[i] <= right[j]:
            result.append(left[i])
            i += 1
        else:
            result.append(right[j])
            j += 1

    # Append any remaining elements from either array
    result.extend(left[i:])
    result.extend(right[j:])

    return result
```
This implementation uses a top-down approach, recursively splitting the input array into smaller halves until each half has only one element, and then merging the sorted halves back together. The `merge` function is used to merge two sorted arrays into a single sorted array.

Note that this implementation has a time complexity of O(n log n) and a space complexity of O(n), making it suitable for large datasets. Additionally, the implementation is stable, meaning that the order of equal elements is preserved.

If you have any critiques or suggestions for improvement, please let me know and I'll be happy to revise the implementation!

In [31]:
# Test the implementation
print("\nTesting the implementation:")
test_code = f"""
{final_response}

if __name__ == "__main__":
    test_cases = [
        [],
        [1],
        [5, 2],
        [3, 1, 4, 1, 5, 9, 2, 6],
        [9, 8, 7, 6, 5, 4, 3, 2, 1],
        [-4, 2, 0, -1, 5]
    ]
    
    for arr in test_cases:
        sorted_arr = merge_sort(arr)
        print(f"Original: {{arr}} => Sorted: {{sorted_arr}}")
"""

display_markdown(f"```python\n{test_code}\n```", raw=True)


Testing the implementation:


```python

Here is a production-quality Python implementation of the Merge Sort algorithm:
```
def merge_sort(arr: list) -> list:
    """
    Sorts an array of elements using the Merge Sort algorithm.

    Time complexity: O(n log n)
    Space complexity: O(n)

    :param arr: The input array to be sorted
    :return: A new sorted array
    """
    # Base case: If the input array has only one element, it is already sorted
    if len(arr) <= 1:
        return arr

    # Split the input array into two halves
    mid = len(arr) // 2
    left = arr[:mid]
    right = arr[mid:]

    # Recursively sort the two halves
    left = merge_sort(left)
    right = merge_sort(right)

    # Merge the two sorted halves into a single sorted array
    return merge(left, right)


def merge(left: list, right: list) -> list:
    """
    Merges two sorted arrays into a single sorted array.

    :param left: The first sorted array
    :param right: The second sorted array
    :return: A new sorted array containing all elements from both input arrays
    """
    result = []
    i = j = 0

    # Merge smaller elements from both arrays
    while i < len(left) and j < len(right):
        if left[i] <= right[j]:
            result.append(left[i])
            i += 1
        else:
            result.append(right[j])
            j += 1

    # Append any remaining elements from either array
    result.extend(left[i:])
    result.extend(right[j:])

    return result
```
This implementation uses a top-down approach, recursively splitting the input array into smaller halves until each half has only one element, and then merging the sorted halves back together. The `merge` function is used to merge two sorted arrays into a single sorted array.

Note that this implementation has a time complexity of O(n log n) and a space complexity of O(n), making it suitable for large datasets. Additionally, the implementation is stable, meaning that the order of equal elements is preserved.

If you have any critiques or suggestions for improvement, please let me know and I'll be happy to revise the implementation!

if __name__ == "__main__":
    test_cases = [
        [],
        [1],
        [5, 2],
        [3, 1, 4, 1, 5, 9, 2, 6],
        [9, 8, 7, 6, 5, 4, 3, 2, 1],
        [-4, 2, 0, -1, 5]
    ]
    
    for arr in test_cases:
        sorted_arr = merge_sort(arr)
        print(f"Original: {arr} => Sorted: {sorted_arr}")

```